Connect Google Colab

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#import os
#os.chdir('/content/drive/MyDrive/...')

Import package

In [11]:
import pandas as pd
import numpy as np

import re
import ast

from tqdm import tqdm
tqdm.pandas()

json to csv

In [4]:
# training set
json_filename = 'NQuAD_train.json'
df = pd.read_json(json_filename)

csv_filename = 'NQuAD_train.csv'
df.to_csv(csv_filename, encoding='utf-8-sig', index=False)

In [3]:
# test set
json_filename = 'NQuAD_test.json'
df = pd.read_json(json_filename)

csv_filename = 'NQuAD_test.csv'
df.to_csv(csv_filename, encoding='utf-8-sig', index=False)

Read data

In [5]:
# read train data
data_train = pd.read_csv('NQuAD_train.csv')
data_train.head()

,news_article,question_stem,answer_options,ans,target_num,sentences_containing_the_numeral_in_answer_options
0,['宏達電（2498）昨(21)日股東會上，董事長王雪紅應允小股東請求，自掏腰包贈送股東會議...,王雪紅贈股東___G新One，創台股東會單價最高贈品 ...,"['100', '2.19', '21', '32']",3,32,"[['當日符合條件股東數約100多位'], ['單機市價約2.19萬元', '依單機2.19..."
1,['印刷電板大廠燿華(2367)於今(24)日召開股東常會，會議順利結束，議程照案通過(見圖...,燿華：Q___獲利有很大改善，Q3至少二位數成長 ...,"['0.88', '2', '5.06', '7.6']",1,2,"[['每股虧損0.88元'], ['至少會有2位數的成長'], ['稅後虧損5.06億元']..."
2,['印刷電板大廠燿華(2367)於今(24)日召開股東常會，會議順利結束，議程照案通過(見圖...,燿華：Q2獲利有很大改善，Q___至少二位數成長 ...,"['0.88', '2', '5.06', '7.6']",1,3,"[['每股虧損0.88元'], ['至少會有2位數的成長'], ['稅後虧損5.06億元']..."
3,['歐洲電信業巨擘Vodafone集團，24日提出以77億歐元(101億美元)收購德國最大有...,Vodafone擬拓展營運 出價___億歐元收購德有線電視 ...,"['101', '75', '77', '84.50']",2,77,[['24日提出以77億歐元(101億美元)收購德國最大有線電視業者Kabel Deutsc...
4,['大陸展訊通信(SPRD.US)宣布，收到大陸清華大學旗下紫光集團發出的13.8億美元收購...,《陸企》展訊擬借紫光收購回歸A股，股價漲逾___% ...,"['13.8', '14', '16.24', '21']",2,16,"[['收到大陸清華大學旗下紫光集團發出的13.8億美元收購要約'], ['展訊通信6月14日..."


In [6]:
# read test data
data_test = pd.read_csv('NQuAD_test.csv')
data_test.head()

,news_article,question_stem,answer_options,ans,target_num,sentences_containing_the_numeral_in_answer_options
0,['日本蘋果情報網站iPhone Mania 23日報導，據調查公司IHS Markit公布...,iPhone 6s為全球最暢銷智慧機；OPPO A53勝過三星S___ ...,"['6', '6.0', '7', '7.2']",2,7,"[['而是前一世代的iPhone 6s', '除iPhone 6s', 'iPhone 6s..."
1,['建準(2421)受惠於美系NB出貨動能延續，前2月合併營收年增15%，3月隨著工作天數回...,建準___月營收動能轉強，Q2展望仍正面 ...,"['2', '2.28', '3', '4']",2,3,"[['前2月合併營收年增15%', '而Q2在NB', 'Q2營收仍可維持年增表現', '建..."
2,['建準(2421)受惠於美系NB出貨動能延續，前2月合併營收年增15%，3月隨著工作天數回...,建準3月營收動能轉強，Q___展望仍正面 ...,"['1.77', '2', '2.28', '3']",1,2,"[['較2015年的1.77元大增'], ['前2月合併營收年增15%', '而Q2在NB'..."
3,['美國能源資訊局（EIA）3月22日公佈，截至3月17日為止的上週乙醇日均產量較前週104...,美國上週乙醇產量略減 去年出口逾___億加侖 ...,"['10', '10.5', '7', '9.9']",0,10,"[['美國乙醇日產量自去年10月底以來均維持在百萬桶以上', '中西部地區的乙醇庫存量較前週..."
4,['面板廠瀚宇彩晶（6116）轉投資的觸控面板廠和鑫光電（3049）今年1、2月因農曆春節影...,和鑫Q___營收估近上季；Q2可望持穩 ...,"['0.48', '0.67', '1', '1.78']",2,1,"[['每股盈餘0.48元'], ['每股虧損0.67元'], ['面板廠瀚宇彩晶（6116）..."


Pre-processing

In [12]:
# training set
data_train['label'] = data_train['ans']
data_train['article'] = data_train['sentences_containing_the_numeral_in_answer_options'] + "," + data_train['question_stem'] + "," + ""+data_train['answer_options']

# test set
data_test['label'] = data_test['ans']
data_test['article'] = data_test['sentences_containing_the_numeral_in_answer_options'] + "," + data_test['question_stem'] + "," + ""+data_test['answer_options']

Symbol Removing

In [13]:
def rm_link(text):
  return re.sub(r'https?://\S+|www\.\S+', '', text)

def rm_punct2(text):
  return re.sub(r'[\"\#\&\'\(\)\*\:\;\=\@\[\\\]\^\`\{\|\}\~]', ' ', text)

def rm_html(text):
  return re.sub(r'<[^>]+>', '', text)

def space_bt_punct(text):
  pattern = r'([,!?-])'
  s = re.sub(pattern, r' \1 ', text)     # add whitespaces between punctuation
  s = re.sub(r'\s{2,}', ' ', s)        # remove double whitespaces
  return s

def rm_whitespaces(text):
  return re.sub(r' +', ' ', text)

def rm_comma_in_num(text):
  return re.sub(r'(\d+),(\d+)', r'\1\2', text)

def clean_pipeline(text):
  no_link = rm_link(text)
  no_comma_num = rm_comma_in_num(no_link)
  space_punct = space_bt_punct(no_comma_num)
  no_punct = rm_punct2(space_punct)
  no_whitespaces = rm_whitespaces(no_punct)
  return no_whitespaces

Stop-word Removal

In [14]:
# read stop-word list
with open('scu_stopwords.txt', 'r', encoding='utf-8') as file:
  lines = file.readlines()
print(lines)

# remove \n
lines = [line.strip() for line in lines]
print(lines)

['打开天窗说亮话\n', '到目前为止\n', '赶早不赶晚\n', '常言说得好\n', '何乐而不为\n', '毫无保留地\n', '由此可见\n', '这就是说\n', '这么点儿\n', '综上所述\n', '总的来看\n', '总的来说\n', '总的说来\n', '总而言之\n', '相对而言\n', '除此之外\n', '反过来说\n', '恰恰相反\n', '如上所述\n', '换句话说\n', '具体地说\n', '具体说来\n', '另一方面\n', '与此同时\n', '一则通过\n', '毫无例外\n', '不然的话\n', '从此以后\n', '从古到今\n', '从古至今\n', '从今以后\n', '大张旗鼓\n', '从无到有\n', '从早到晚\n', '弹指之间\n', '不亦乐乎\n', '不知不觉\n', '不止一次\n', '不择手段\n', '不可开交\n', '不可抗拒\n', '不仅仅是\n', '不管怎样\n', '挨家挨户\n', '长此下去\n', '长话短说\n', '除此而外\n', '除此以外\n', '除此之外\n', '得天独厚\n', '川流不息\n', '长期以来\n', '挨门挨户\n', '挨门逐户\n', '多多少少\n', '多多益善\n', '二话不说\n', '更进一步\n', '二话没说\n', '分期分批\n', '风雨无阻\n', '归根到底\n', '归根结底\n', '反之亦然\n', '大面儿上\n', '倒不如说\n', '成年累月\n', '换句话说\n', '或多或少\n', '简而言之\n', '接连不断\n', '尽如人意\n', '尽心竭力\n', '尽心尽力\n', '尽管如此\n', '据我所知\n', '具体地说\n', '具体来说\n', '具体说来\n', '近几年来\n', '每时每刻\n', '屡次三番\n', '三番两次\n', '三番五次\n', '三天两头\n', '另一方面\n', '老老实实\n', '年复一年\n', '恰恰相反\n', '顷刻之间\n', '穷年累月\n', '千万千万\n', '日复一日\n', '如此等等\n', '如前所述\n', '如上所述\n', '一方面\n', '切不可\n', '顷刻间\n', '全身心

In [15]:
%pip install opencc-python-reimplemented

   ---------------------------------------- 0.0/481.8 kB ? eta -:--:--
    --------------------------------------- 10.2/481.8 kB ? eta -:--:--
   -- ------------------------------------ 30.7/481.8 kB 435.7 kB/s eta 0:00:02
   ----- --------------------------------- 71.7/481.8 kB 558.5 kB/s eta 0:00:01
   ----------- -------------------------- 143.4/481.8 kB 847.9 kB/s eta 0:00:01
   ----------------------- ---------------- 286.7/481.8 kB 1.4 MB/s eta 0:00:01
   -------------------------------------- - 460.8/481.8 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------  481.3/481.8 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 481.8/481.8 kB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [16]:
# translation from simplified Chinese to traditional Chinese
from opencc import OpenCC

config_file_path = 's2twp'
converter = OpenCC(config_file_path)

stopwords_list = [converter.convert(text) for text in lines]
print(stopwords_list)

['開啟天窗說亮話', '到目前為止', '趕早不趕晚', '常言說得好', '何樂而不為', '毫無保留地', '由此可見', '這就是說', '這麼點兒', '綜上所述', '總的來看', '總的來說', '總的說來', '總而言之', '相對而言', '除此之外', '反過來說', '恰恰相反', '如上所述', '換句話說', '具體地說', '具體說來', '另一方面', '與此同時', '一則透過', '毫無例外', '不然的話', '從此以後', '從古到今', '從古至今', '從今以後', '大張旗鼓', '從無到有', '從早到晚', '彈指之間', '不亦樂乎', '不知不覺', '不止一次', '不擇手段', '不可開交', '不可抗拒', '不僅僅是', '不管怎樣', '挨家挨戶', '長此下去', '長話短說', '除此而外', '除此以外', '除此之外', '得天獨厚', '川流不息', '長期以來', '挨門挨戶', '挨門逐戶', '多多少少', '多多益善', '二話不說', '更進一步', '二話沒說', '分期分批', '風雨無阻', '歸根到底', '歸根結底', '反之亦然', '大面兒上', '倒不如說', '成年累月', '換句話說', '或多或少', '簡而言之', '接連不斷', '盡如人意', '盡心竭力', '盡心盡力', '儘管如此', '據我所知', '具體地說', '具體來說', '具體說來', '近幾年來', '每時每刻', '屢次三番', '三番兩次', '三番五次', '三天兩頭', '另一方面', '老老實實', '年復一年', '恰恰相反', '頃刻之間', '窮年累月', '千萬千萬', '日復一日', '如此等等', '如前所述', '如上所述', '一方面', '切不可', '頃刻間', '全身心', '另方面', '另一個', '猛然間', '默默地', '就是說', '近年來', '儘可能', '接下來', '簡言之', '急匆匆', '即是說', '基本上', '換言之', '充其極', '充其量', '暗地裡', '反之則', '比如說', '背地裡', '背靠背', '並沒有', '不得不', '不得了', '不得已', '不僅僅', '不經意', '不能不', '不外乎',

In [17]:
# use tokenizer jieba
import jieba

# preprocessing
def tokenize(text):
  return list(jieba.cut(text))

def rm_stopwords(text, stopwords_list):
  return ''.join([word for word in text if word not in stopwords_list])

def preprocess_pipeline(text):
  tokens = tokenize(text)
  no_stopwords = rm_stopwords(tokens, stopwords_list)
  return no_stopwords

In [18]:
# data preview
str1 = rm_link(data_train['article'][0])
print(str1)
str1 = rm_comma_in_num(str1)
print(str1)
str1 = space_bt_punct(str1)
print(str1)
str1 = rm_punct2(str1)
print(str1)
str1 = rm_whitespaces(str1)
print(str1)

print()
str1 = tokenize(str1)
print(str1)
str1 = rm_stopwords(str1, stopwords_list)
print(str1)
str1 = rm_whitespaces(str1)
print(str1)

Building prefix dict from the default dictionary ...


[['當日符合條件股東數約100多位'], ['單機市價約2.19萬元', '依單機2.19萬元計算'], ['宏達電（2498）昨(21)日股東會上'], ['每人一台宏達電旗艦手機新HTC One 32G手機']],王雪紅贈股東___G新One，創台股東會單價最高贈品                                                       ,['100', '2.19', '21', '32']
[['當日符合條件股東數約100多位'], ['單機市價約2.19萬元', '依單機2.19萬元計算'], ['宏達電（2498）昨(21)日股東會上'], ['每人一台宏達電旗艦手機新HTC One 32G手機']],王雪紅贈股東___G新One，創台股東會單價最高贈品                                                       ,['100', '2.19', '21', '32']
[['當日符合條件股東數約100多位'] , ['單機市價約2.19萬元' , '依單機2.19萬元計算'] , ['宏達電（2498）昨(21)日股東會上'] , ['每人一台宏達電旗艦手機新HTC One 32G手機']] , 王雪紅贈股東___G新One，創台股東會單價最高贈品 , ['100' , '2.19' , '21' , '32']
   當日符合條件股東數約100多位   ,   單機市價約2.19萬元  ,  依單機2.19萬元計算   ,   宏達電（2498）昨 21 日股東會上   ,   每人一台宏達電旗艦手機新HTC One 32G手機    , 王雪紅贈股東___G新One，創台股東會單價最高贈品 ,   100  ,  2.19  ,  21  ,  32  
 當日符合條件股東數約100多位 , 單機市價約2.19萬元 , 依單機2.19萬元計算 , 宏達電（2498）昨 21 日股東會上 , 每人一台宏達電旗艦手機新HTC One 32G手機 , 王雪紅贈股東___G新One，創台股東會單價最高贈品 , 100 , 2.19 , 21 , 32 



Dumping model to file cache C:\Users\abcd7\AppData\Local\Temp\jieba.cache
Loading model cost 0.617 seconds.
Prefix dict has been built successfully.


[' ', '當日', '符合', '條件', '股東數', '約', '100', '多位', ' ', ',', ' ', '單機市', '價約', '2.19', '萬元', ' ', ',', ' ', '依單機', '2.19', '萬元', '計算', ' ', ',', ' ', '宏達電', '（', '2498', '）', '昨', ' ', '21', ' ', '日股', '東會', '上', ' ', ',', ' ', '每人', '一台', '宏達', '電旗', '艦手', '機新', 'HTC', ' ', 'One', ' ', '32G', '手機', ' ', ',', ' ', '王雪紅贈', '股東', '___', 'G', '新', 'One', '，', '創台', '股東會', '單價', '最高', '贈品', ' ', ',', ' ', '100', ' ', ',', ' ', '2.19', ' ', ',', ' ', '21', ' ', ',', ' ', '32', ' ']
 當日符合條件股東數約100多位 , 單機市價約2.19萬元 , 依單機2.19萬元計算 , 宏達電（2498）昨 21 日股東會上 , 每人一台宏達電旗艦手機新HTC One 32G手機 , 王雪紅贈股東___G新One，創台股東會單價最高贈品 , 100 , 2.19 , 21 , 32 
 當日符合條件股東數約100多位 , 單機市價約2.19萬元 , 依單機2.19萬元計算 , 宏達電（2498）昨 21 日股東會上 , 每人一台宏達電旗艦手機新HTC One 32G手機 , 王雪紅贈股東___G新One，創台股東會單價最高贈品 , 100 , 2.19 , 21 , 32 


In [19]:
# training set
# Symbol Removing
data_train['clean'] = data_train['article'].progress_apply(clean_pipeline)
# Stop-word Removal
data_train['processed'] = data_train['clean'].progress_apply(preprocess_pipeline)

data_train.head()

100%|██████████| 57750/57750 [01:56<00:00, 496.79it/s]


,news_article,question_stem,answer_options,ans,target_num,sentences_containing_the_numeral_in_answer_options,label,article,clean,processed
0,['宏達電（2498）昨(21)日股東會上，董事長王雪紅應允小股東請求，自掏腰包贈送股東會議...,王雪紅贈股東___G新One，創台股東會單價最高贈品 ...,"['100', '2.19', '21', '32']",3,32,"[['當日符合條件股東數約100多位'], ['單機市價約2.19萬元', '依單機2.19...",3,"[['當日符合條件股東數約100多位'], ['單機市價約2.19萬元', '依單機2.19...","當日符合條件股東數約100多位 , 單機市價約2.19萬元 , 依單機2.19萬元計算 ,...","當日符合條件股東數約100多位 , 單機市價約2.19萬元 , 依單機2.19萬元計算 ,..."
1,['印刷電板大廠燿華(2367)於今(24)日召開股東常會，會議順利結束，議程照案通過(見圖...,燿華：Q___獲利有很大改善，Q3至少二位數成長 ...,"['0.88', '2', '5.06', '7.6']",1,2,"[['每股虧損0.88元'], ['至少會有2位數的成長'], ['稅後虧損5.06億元']...",1,"[['每股虧損0.88元'], ['至少會有2位數的成長'], ['稅後虧損5.06億元']...","每股虧損0.88元 , 至少會有2位數的成長 , 稅後虧損5.06億元 , 毛利率為7.6...","每股虧損0.88元 , 至少有2位數的成長 , 稅後虧損5.06億元 , 毛利率為7.6%..."
2,['印刷電板大廠燿華(2367)於今(24)日召開股東常會，會議順利結束，議程照案通過(見圖...,燿華：Q2獲利有很大改善，Q___至少二位數成長 ...,"['0.88', '2', '5.06', '7.6']",1,3,"[['每股虧損0.88元'], ['至少會有2位數的成長'], ['稅後虧損5.06億元']...",1,"[['每股虧損0.88元'], ['至少會有2位數的成長'], ['稅後虧損5.06億元']...","每股虧損0.88元 , 至少會有2位數的成長 , 稅後虧損5.06億元 , 毛利率為7.6...","每股虧損0.88元 , 至少有2位數的成長 , 稅後虧損5.06億元 , 毛利率為7.6%..."
3,['歐洲電信業巨擘Vodafone集團，24日提出以77億歐元(101億美元)收購德國最大有...,Vodafone擬拓展營運 出價___億歐元收購德有線電視 ...,"['101', '75', '77', '84.50']",2,77,[['24日提出以77億歐元(101億美元)收購德國最大有線電視業者Kabel Deutsc...,2,[['24日提出以77億歐元(101億美元)收購德國最大有線電視業者Kabel Deutsc...,24日提出以77億歐元 101億美元 收購德國最大有線電視業者Kabel Deutschl...,24日提出以77億歐元 101億美元 收購德國最大有線電視業者Kabel Deutschl...
4,['大陸展訊通信(SPRD.US)宣布，收到大陸清華大學旗下紫光集團發出的13.8億美元收購...,《陸企》展訊擬借紫光收購回歸A股，股價漲逾___% ...,"['13.8', '14', '16.24', '21']",2,16,"[['收到大陸清華大學旗下紫光集團發出的13.8億美元收購要約'], ['展訊通信6月14日...",2,"[['收到大陸清華大學旗下紫光集團發出的13.8億美元收購要約'], ['展訊通信6月14日...","收到大陸清華大學旗下紫光集團發出的13.8億美元收購要約 , 展訊通信6月14日宣布提高第...","收到大陸清華大學旗下紫光集團發出的13.8億美元收購要約 , 展訊通信6月14日宣布提高第..."


In [20]:
# test set
# Symbol Removing
data_test['clean'] = data_test['article'].progress_apply(clean_pipeline)
# Stop-word Removal
data_test['processed'] = data_test['clean'].progress_apply(preprocess_pipeline)

data_test.head()

100%|██████████| 14438/14438 [00:27<00:00, 519.17it/s]


,news_article,question_stem,answer_options,ans,target_num,sentences_containing_the_numeral_in_answer_options,label,article,clean,processed
0,['日本蘋果情報網站iPhone Mania 23日報導，據調查公司IHS Markit公布...,iPhone 6s為全球最暢銷智慧機；OPPO A53勝過三星S___ ...,"['6', '6.0', '7', '7.2']",2,7,"[['而是前一世代的iPhone 6s', '除iPhone 6s', 'iPhone 6s...",2,"[['而是前一世代的iPhone 6s', '除iPhone 6s', 'iPhone 6s...","而是前一世代的iPhone 6s , 除iPhone 6s , iPhone 6s Plu...","前一世代的iPhone 6s , 除iPhone 6s , iPhone 6s Plus分..."
1,['建準(2421)受惠於美系NB出貨動能延續，前2月合併營收年增15%，3月隨著工作天數回...,建準___月營收動能轉強，Q2展望仍正面 ...,"['2', '2.28', '3', '4']",2,3,"[['前2月合併營收年增15%', '而Q2在NB', 'Q2營收仍可維持年增表現', '建...",2,"[['前2月合併營收年增15%', '而Q2在NB', 'Q2營收仍可維持年增表現', '建...","前2月合併營收年增15% , 而Q2在NB , Q2營收仍可維持年增表現 , 建準今年擬配...","前2月合併營收年增15% , 而Q2在NB , Q2營收仍可維持年增表現 , 建準今年擬配..."
2,['建準(2421)受惠於美系NB出貨動能延續，前2月合併營收年增15%，3月隨著工作天數回...,建準3月營收動能轉強，Q___展望仍正面 ...,"['1.77', '2', '2.28', '3']",1,2,"[['較2015年的1.77元大增'], ['前2月合併營收年增15%', '而Q2在NB'...",1,"[['較2015年的1.77元大增'], ['前2月合併營收年增15%', '而Q2在NB'...","較2015年的1.77元大增 , 前2月合併營收年增15% , 而Q2在NB , Q2營收...","較2015年的1.77元大增 , 前2月合併營收年增15% , 而Q2在NB , Q2營收..."
3,['美國能源資訊局（EIA）3月22日公佈，截至3月17日為止的上週乙醇日均產量較前週104...,美國上週乙醇產量略減 去年出口逾___億加侖 ...,"['10', '10.5', '7', '9.9']",0,10,"[['美國乙醇日產量自去年10月底以來均維持在百萬桶以上', '中西部地區的乙醇庫存量較前週...",0,"[['美國乙醇日產量自去年10月底以來均維持在百萬桶以上', '中西部地區的乙醇庫存量較前週...","美國乙醇日產量自去年10月底以來均維持在百萬桶以上 , 中西部地區的乙醇庫存量較前週增加1...","美國乙醇日產量自去年10月底以來均維持在百萬桶以上 , 中西部地區的乙醇庫存量較前週增加1..."
4,['面板廠瀚宇彩晶（6116）轉投資的觸控面板廠和鑫光電（3049）今年1、2月因農曆春節影...,和鑫Q___營收估近上季；Q2可望持穩 ...,"['0.48', '0.67', '1', '1.78']",2,1,"[['每股盈餘0.48元'], ['每股虧損0.67元'], ['面板廠瀚宇彩晶（6116）...",2,"[['每股盈餘0.48元'], ['每股虧損0.67元'], ['面板廠瀚宇彩晶（6116）...","每股盈餘0.48元 , 每股虧損0.67元 , 面板廠瀚宇彩晶（6116）轉投資的觸控面板...","每股盈餘0.48元 , 每股虧損0.67元 , 面板廠瀚宇彩晶（6116）轉投資的觸控面板..."


Numerals Augmentation and Feature Enhancing

In [22]:
def replace_numural(sentence, option):
  nums = ast.literal_eval(option)
  nums = [re.escape(str(num)) for num in nums]
  #remove "," in option
  nums = [re.sub(r',', '', str(num)) for num in nums]

  result = re.sub(rf'(?<!\$)(?<![\d.]){nums[3]}(?!\$)', '$3$', sentence)
  #print(result)
  result = re.sub(rf'(?<!\$)(?<![\d.]){nums[2]}(?!\$)', '$2$', result)
  #print(result)
  result = re.sub(rf'(?<!\$)(?<![\d.]){nums[1]}(?!\$)', '$1$', result)
  #print(result)
  result = re.sub(rf'(?<!\$)(?<![\d.]){nums[0]}(?!\$)', '$0$', result)
  #print(result)

  return result

Demo

In [23]:
data_train['processed'][0]

' 當日符合條件股東數約100多位 , 單機市價約2.19萬元 , 依單機2.19萬元計算 , 宏達電（2498）昨 21 日股東會上 , 每人一台宏達電旗艦手機新HTC One 32G手機 , 王雪紅贈股東___G新One，創台股東會單價最高贈品 , 100 , 2.19 , 21 , 32 '

In [24]:
replace_numural(data_train['processed'][0], data_train['answer_options'][0])

' 當日符合條件股東數約$0$多位 , 單機市價約$1$萬元 , 依單機$1$萬元計算 , 宏達電（2498）昨 $2$ 日股東會上 , 每人一台宏達電旗艦手機新HTC One $3$G手機 , 王雪紅贈股東___G新One，創台股東會單價最高贈品 , $0$ , $1$ , $2$ , $3$ '

In [ ]:
# NA and FE for training set
data_train['rm_numural'] = ''

for i in tqdm(range(data_train.shape[0]), desc="Processing"):
  data_train['rm_numural'][i] = replace_numural(data_train['processed'][i], data_train['answer_options'][i])

In [27]:
data_train.head()

,news_article,question_stem,answer_options,ans,target_num,sentences_containing_the_numeral_in_answer_options,label,article,clean,processed,rm_numural
0,['宏達電（2498）昨(21)日股東會上，董事長王雪紅應允小股東請求，自掏腰包贈送股東會議...,王雪紅贈股東___G新One，創台股東會單價最高贈品 ...,"['100', '2.19', '21', '32']",3,32,"[['當日符合條件股東數約100多位'], ['單機市價約2.19萬元', '依單機2.19...",3,"[['當日符合條件股東數約100多位'], ['單機市價約2.19萬元', '依單機2.19...","當日符合條件股東數約100多位 , 單機市價約2.19萬元 , 依單機2.19萬元計算 ,...","當日符合條件股東數約100多位 , 單機市價約2.19萬元 , 依單機2.19萬元計算 ,...","當日符合條件股東數約$0$多位 , 單機市價約$1$萬元 , 依單機$1$萬元計算 , 宏..."
1,['印刷電板大廠燿華(2367)於今(24)日召開股東常會，會議順利結束，議程照案通過(見圖...,燿華：Q___獲利有很大改善，Q3至少二位數成長 ...,"['0.88', '2', '5.06', '7.6']",1,2,"[['每股虧損0.88元'], ['至少會有2位數的成長'], ['稅後虧損5.06億元']...",1,"[['每股虧損0.88元'], ['至少會有2位數的成長'], ['稅後虧損5.06億元']...","每股虧損0.88元 , 至少會有2位數的成長 , 稅後虧損5.06億元 , 毛利率為7.6...","每股虧損0.88元 , 至少有2位數的成長 , 稅後虧損5.06億元 , 毛利率為7.6%...","每股虧損$0$元 , 至少有$1$位數的成長 , 稅後虧損$2$億元 , 毛利率為$3$%..."
2,['印刷電板大廠燿華(2367)於今(24)日召開股東常會，會議順利結束，議程照案通過(見圖...,燿華：Q2獲利有很大改善，Q___至少二位數成長 ...,"['0.88', '2', '5.06', '7.6']",1,3,"[['每股虧損0.88元'], ['至少會有2位數的成長'], ['稅後虧損5.06億元']...",1,"[['每股虧損0.88元'], ['至少會有2位數的成長'], ['稅後虧損5.06億元']...","每股虧損0.88元 , 至少會有2位數的成長 , 稅後虧損5.06億元 , 毛利率為7.6...","每股虧損0.88元 , 至少有2位數的成長 , 稅後虧損5.06億元 , 毛利率為7.6%...","每股虧損$0$元 , 至少有$1$位數的成長 , 稅後虧損$2$億元 , 毛利率為$3$%..."
3,['歐洲電信業巨擘Vodafone集團，24日提出以77億歐元(101億美元)收購德國最大有...,Vodafone擬拓展營運 出價___億歐元收購德有線電視 ...,"['101', '75', '77', '84.50']",2,77,[['24日提出以77億歐元(101億美元)收購德國最大有線電視業者Kabel Deutsc...,2,[['24日提出以77億歐元(101億美元)收購德國最大有線電視業者Kabel Deutsc...,24日提出以77億歐元 101億美元 收購德國最大有線電視業者Kabel Deutschl...,24日提出以77億歐元 101億美元 收購德國最大有線電視業者Kabel Deutschl...,24日提出以$2$億歐元 $0$億美元 收購德國最大有線電視業者Kabel Deutsch...
4,['大陸展訊通信(SPRD.US)宣布，收到大陸清華大學旗下紫光集團發出的13.8億美元收購...,《陸企》展訊擬借紫光收購回歸A股，股價漲逾___% ...,"['13.8', '14', '16.24', '21']",2,16,"[['收到大陸清華大學旗下紫光集團發出的13.8億美元收購要約'], ['展訊通信6月14日...",2,"[['收到大陸清華大學旗下紫光集團發出的13.8億美元收購要約'], ['展訊通信6月14日...","收到大陸清華大學旗下紫光集團發出的13.8億美元收購要約 , 展訊通信6月14日宣布提高第...","收到大陸清華大學旗下紫光集團發出的13.8億美元收購要約 , 展訊通信6月14日宣布提高第...","收到大陸清華大學旗下紫光集團發出的$0$億美元收購要約 , 展訊通信6月$1$日宣布提高第..."


In [ ]:
# NA and FE for test set
data_test['rm_numural'] = ''

for i in tqdm(range(data_test.shape[0]), desc="Processing"):
  data_test['rm_numural'][i] = replace_numural(data_test['processed'][i], data_test['answer_options'][i])

In [28]:
data_test.head()

,news_article,question_stem,answer_options,ans,target_num,sentences_containing_the_numeral_in_answer_options,label,article,clean,processed,rm_numural
0,['日本蘋果情報網站iPhone Mania 23日報導，據調查公司IHS Markit公布...,iPhone 6s為全球最暢銷智慧機；OPPO A53勝過三星S___ ...,"['6', '6.0', '7', '7.2']",2,7,"[['而是前一世代的iPhone 6s', '除iPhone 6s', 'iPhone 6s...",2,"[['而是前一世代的iPhone 6s', '除iPhone 6s', 'iPhone 6s...","而是前一世代的iPhone 6s , 除iPhone 6s , iPhone 6s Plu...","前一世代的iPhone 6s , 除iPhone 6s , iPhone 6s Plus分...","前一世代的iPhone $0$s , 除iPhone $0$s , iPhone $0$s..."
1,['建準(2421)受惠於美系NB出貨動能延續，前2月合併營收年增15%，3月隨著工作天數回...,建準___月營收動能轉強，Q2展望仍正面 ...,"['2', '2.28', '3', '4']",2,3,"[['前2月合併營收年增15%', '而Q2在NB', 'Q2營收仍可維持年增表現', '建...",2,"[['前2月合併營收年增15%', '而Q2在NB', 'Q2營收仍可維持年增表現', '建...","前2月合併營收年增15% , 而Q2在NB , Q2營收仍可維持年增表現 , 建準今年擬配...","前2月合併營收年增15% , 而Q2在NB , Q2營收仍可維持年增表現 , 建準今年擬配...","前$0$月合併營收年增15% , 而Q$0$在NB , Q$0$營收仍可維持年增表現 , ..."
2,['建準(2421)受惠於美系NB出貨動能延續，前2月合併營收年增15%，3月隨著工作天數回...,建準3月營收動能轉強，Q___展望仍正面 ...,"['1.77', '2', '2.28', '3']",1,2,"[['較2015年的1.77元大增'], ['前2月合併營收年增15%', '而Q2在NB'...",1,"[['較2015年的1.77元大增'], ['前2月合併營收年增15%', '而Q2在NB'...","較2015年的1.77元大增 , 前2月合併營收年增15% , 而Q2在NB , Q2營收...","較2015年的1.77元大增 , 前2月合併營收年增15% , 而Q2在NB , Q2營收...","較$1$015年的$0$元大增 , 前$1$月合併營收年增15% , 而Q$1$在NB ,..."
3,['美國能源資訊局（EIA）3月22日公佈，截至3月17日為止的上週乙醇日均產量較前週104...,美國上週乙醇產量略減 去年出口逾___億加侖 ...,"['10', '10.5', '7', '9.9']",0,10,"[['美國乙醇日產量自去年10月底以來均維持在百萬桶以上', '中西部地區的乙醇庫存量較前週...",0,"[['美國乙醇日產量自去年10月底以來均維持在百萬桶以上', '中西部地區的乙醇庫存量較前週...","美國乙醇日產量自去年10月底以來均維持在百萬桶以上 , 中西部地區的乙醇庫存量較前週增加1...","美國乙醇日產量自去年10月底以來均維持在百萬桶以上 , 中西部地區的乙醇庫存量較前週增加1...","美國乙醇日產量自去年$0$月底以來均維持在百萬桶以上 , 中西部地區的乙醇庫存量較前週增加..."
4,['面板廠瀚宇彩晶（6116）轉投資的觸控面板廠和鑫光電（3049）今年1、2月因農曆春節影...,和鑫Q___營收估近上季；Q2可望持穩 ...,"['0.48', '0.67', '1', '1.78']",2,1,"[['每股盈餘0.48元'], ['每股虧損0.67元'], ['面板廠瀚宇彩晶（6116）...",2,"[['每股盈餘0.48元'], ['每股虧損0.67元'], ['面板廠瀚宇彩晶（6116）...","每股盈餘0.48元 , 每股虧損0.67元 , 面板廠瀚宇彩晶（6116）轉投資的觸控面板...","每股盈餘0.48元 , 每股虧損0.67元 , 面板廠瀚宇彩晶（6116）轉投資的觸控面板...","每股盈餘$0$元 , 每股虧損$1$元 , 面板廠瀚宇彩晶（6116）轉投資的觸控面板廠和..."


Save pre-processed Data

In [46]:
data_train[['rm_numural', 'label']].to_csv('./train.csv', index=False, header=True)
data_test[['rm_numural', 'label']].to_csv('./test.csv', index=False, header=True)

df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

df_train = df_train.rename(columns={'rm_numural': 'article'})
df_test = df_test.rename(columns={'rm_numural': 'article'})

df_train[['article', 'label']].to_csv('./train.csv', index=False, header=True)
df_test[['article', 'label']].to_csv('./test.csv', index=False, header=True)

In [47]:
df_train.head()

,article,label
0,"當日符合條件股東數約$0$多位 , 單機市價約$1$萬元 , 依單機$1$萬元計算 , 宏...",3
1,"每股虧損$0$元 , 至少有$1$位數的成長 , 稅後虧損$2$億元 , 毛利率為$3$%...",1
2,"每股虧損$0$元 , 至少有$1$位數的成長 , 稅後虧損$2$億元 , 毛利率為$3$%...",1
3,24日提出以$2$億歐元 $0$億美元 收購德國最大有線電視業者Kabel Deutsch...,2
4,"收到大陸清華大學旗下紫光集團發出的$0$億美元收購要約 , 展訊通信6月$1$日宣布提高第...",2


In [48]:
df_test.head()

,article,label
0,"前一世代的iPhone $0$s , 除iPhone $0$s , iPhone $0$s...",2
1,"前$0$月合併營收年增15% , 而Q$0$在NB , Q$0$營收仍可維持年增表現 , ...",2
2,"較$1$015年的$0$元大增 , 前$1$月合併營收年增15% , 而Q$1$在NB ,...",1
3,"美國乙醇日產量自去年$0$月底以來均維持在百萬桶以上 , 中西部地區的乙醇庫存量較前週增加...",0
4,"每股盈餘$0$元 , 每股虧損$1$元 , 面板廠瀚宇彩晶（6116）轉投資的觸控面板廠和...",2
